<a href="https://colab.research.google.com/github/Karthick47v2/mcq-generator/blob/main/word_distractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TODO : Discard similar meaning words


# GITHUB REPO - https://github.com/explosion/sense2vec

# download and import Sense2Vec
!pip install sense2vec
from sense2vec import Sense2Vec

In [4]:
# download and extract pretrained vectors - Here we are using 2015 reddit comments
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
!tar -xf s2v_reddit_2015_md.tar.gz

In [14]:
# load vectors
s2v = Sense2Vec().from_disk("/content/s2v_old")

In [162]:
def word_distractors(query):
  # TAG - NOUN, CARDINAL, NUM, PROPN, VERB
  query = query.lower()
  query = query.replace(' ', '_')
  query = s2v.get_best_sense(query)

  if query in s2v:
    return s2v.most_similar(query, n=4)
  else:
    return None

In [164]:
def change_format(distractors):
  output = []
  for result in distractors:
    res = result[0].split('|')
    res = res[0].replace('_', ' ')
    res = res[0].upper() + res[1:]
    output.append(res)
  return output

In [ ]:
results = change_format(word_distractors("1000"))
print(results)